In [26]:
from GPTAnnotator import GptAnnotation
import os
from docx import Document


In [8]:
def ReadFile(FilePath):

    def format(resume_text):
        formatted_resume = ""
        lines = resume_text.split('\n')
    
        for line in lines:
            if line.strip():  # Ignore empty lines
                formatted_line = ' '.join(line.split())
                formatted_resume += formatted_line + '\n'
    
        return formatted_resume


    
    from PyPDF2 import PdfReader
    # Get the file extension
    file_extension = FilePath.split('.')[-1].lower()

    if file_extension == 'pdf':
        doc_reader = PdfReader(FilePath)
        raw_text = ''
        for i, page in enumerate(doc_reader.pages):
            text = page.extract_text()
            if text:
                raw_text += text

    elif file_extension == 'docx':
        doc = Document(FilePath)
        raw_text = ''
        for paragraph in doc.paragraphs:
            raw_text += paragraph.text + '\n'

    elif file_extension == 'txt':
        with open(FilePath, 'r', encoding='utf-8') as file:
            raw_text = file.read()
            
    elif file_extension == 'md':
        with open(FilePath, 'r', encoding='utf-8') as file:
            md_content = file.read()
            raw_text = markdown.markdown(md_content)

    else:
        raise ValueError(f"Unsupported file format: {file_extension}")

    raw_text = raw_text.replace("\n"," ")
    raw_text = raw_text.replace("  "," ") 

    return format(raw_text)


In [19]:
def create_json_list(resume, annotation):  
    """  
    This function takes a resume and its annotation, and returns a list of entities found in the resume.  
    Each entity is represented as a list containing its text, its tag, and its start and end positions in the resume.  
    """  

    def find_start_end(input_string, substring):  
        """  
        This helper function finds the start and end positions of a substring in a string.  
        If the substring is more than 6 words long, it only uses the first 3 words to find the start position.  
        """  
        words = substring.split()  
  
        if len(words) > 6:  
            substring_start = " ".join(words[:3])  
            start_position = input_string.find(substring_start)  
        else:  
            start_position = input_string.find(substring)  
  
        if start_position != -1:  
            end_position = start_position + len(substring) - 1  
            return start_position, end_position + 1  
        else:  
            return None  
  
    elements = []  
    pos = []
    text = ""

    
  
    # Split the annotation into lines  
    annotation = annotation.split("\n")  
  
    # Extract the entity and tag from each line  
    for line in annotation:  
        try:  
            entity = line.split("'")[1]  
            tag = line.split("'")[2].split("-<")[1].split(">-")[0]  
            elements.append([entity, tag])  
            text+=entity+"\n"
        except IndexError:  
            pass  
  
    # Find the start and end positions of each entity in the resume  
    for element in elements:  
      
        position = find_start_end(resume, element[0])  
        # If the position is not None, add it to the element  
        if position is not None and position not in pos:  
            print(position,element[0])  

            element.append(position)  
            pos.append(position)
        else:
            print(position,element[0])
            #Embedding = GenerateEmbeddings(resumePath)
            #Element = QuerySimilaritySearch(element[0],Embedding,1)
            #print(f"Found{Element}")
            pass
            

  
    # Filter out any elements that did not have a position  
    elements = [element for element in elements if len(element) == 3]  
  
    return elements 

In [10]:
l = [4,5,6,7]
5 in l


True

In [11]:
def CreateJsonText(Resume,AnnotationList):
    input_data	= AnnotationList
    # Prepare your text and annotations  
    text = ""  
    annotations = []  
      
    # Iterate over the input data  
    for entry in input_data:  
        if entry[2] != None:
            annotations.append([entry[2][0], entry[2][1], entry[1]])  
      
    # Put everything into the final format  
    data = [Resume, {"entities": annotations}]

    return data


In [12]:
def get_files_in_folder(folder_path):
    file_paths = []
    
    # Iterate over all files in the folder
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            file_paths.append(file_path)
    
    return file_paths

In [20]:
resume = ReadFile("res/10219099.pdf")
annotation = GptAnnotation(resume, ["NAME", "LOCATION", "EDUCATION", "EXPERIENCE", "SKILLS", "PROJECTS", "COURSES", "LANGUAGES", "CERTIFICATIONS", "PUBLICATIONS", "DATES"])  
for lines in annotation.split("\n"):
    print(lines)

EQUIPMENT ENGINEERING TECHNICIAN Professional Summary SkilledÂ Maintenance MechanicÂ Technician with superb problem solving and multi-tasking skills. Self-directed and motivated worker. To obtain a position where Teamwork, Integrity and Proffesionalism are a high standard in the Company's Mission Statement. All while engaing in new challenges and learning experiences. Skills Â·Â Â 20 years of Material Management experienceÂ with State and Federal Government Â· 13 years of Law Enforcement experience with the United States Marine Corps and Army National Guard. Â·Â Knowledge of mechanical test equipment. Â· Work from written/verbal instructions, schematics, rough sketches, troubleshooting diagrams, layouts and plans, interpretation and application of technical knowledge and understanding of mechanical theory and principles. Â· Proficient in the use of PC's, MSÂ office, 10key by touch and Customer Service skills. Â·Â Forklift and Crane Certified Â·Â Self motivated to Lead and Achieve Compa

In [22]:
AnnotationList = create_json_list(resume, annotation)  
AnnotationList

(1160, 1192) Equipment Engineering Technician
None Skilled Maintenance Mechanic Technician with superb problem solving and multi-tasking skills.
(150, 185) Self-directed and motivated worker.
(309, 370) All while engaing in new challenges and learning experiences.
(384, 460) 20 years of Material Management experience with State and Federal Government
(465, 564) 13 years of Law Enforcement experience with the United States Marine Corps and Army National Guard.
(569, 608) Knowledge of mechanical test equipment.
(612, 832) Work from written/verbal instructions, schematics, rough sketches, troubleshooting diagrams, layouts and plans, interpretation and application of technical knowledge and understanding of mechanical theory and principles.
(927, 955) Forklift and Crane Certified
(960, 1051) Self motivated to Lead and Achieve Company goals with excellence, integrity and efficiency.
(1055, 1067) NDT Level II
(1073, 1143) Knowledge on mechanical, pneumatic and hydraulic theory and principles

[['Equipment Engineering Technician', 'EXPERIENCE', (1160, 1192)],
 ['Self-directed and motivated worker.', 'SKILLS', (150, 185)],
 ['All while engaing in new challenges and learning experiences.',
  'SKILLS',
  (309, 370)],
 ['20 years of Material Management experience with State and Federal Government',
  'SKILLS',
  (384, 460)],
 ['13 years of Law Enforcement experience with the United States Marine Corps and Army National Guard.',
  'SKILLS',
  (465, 564)],
 ['Knowledge of mechanical test equipment.', 'SKILLS', (569, 608)],
 ['Work from written/verbal instructions, schematics, rough sketches, troubleshooting diagrams, layouts and plans, interpretation and application of technical knowledge and understanding of mechanical theory and principles.',
  'SKILLS',
  (612, 832)],
 ['Forklift and Crane Certified', 'SKILLS', (927, 955)],
 ['Self motivated to Lead and Achieve Company goals with excellence, integrity and efficiency.',
  'SKILLS',
  (960, 1051)],
 ['NDT Level II', 'SKILLS', (10

In [34]:
import json  
import os
from tqdm import tqdm
  
def AnnotationMaker(FolderPath):
    files = get_files_in_folder(FolderPath)  
    
    json_data = []  
    total_files = len(files)
    processed_files = 0

    for file in tqdm(files):  # tqdm will automatically create a progress bar
        try:  
            resume = ReadFile(file)  
            annotation = GptAnnotation(resume, ["NAME", "LOCATION", "EDUCATION", "EXPERIENCE", "SKILLS", "PROJECTS", "COURSES", "LANGUAGES", "CERTIFICATIONS", "PUBLICATIONS", "DATES"])  
            AnnotationList = create_json_list(resume, annotation)  
            JsonData = CreateJsonText(resume,AnnotationList)  
            json_data.append(JsonData)
            processed_files += 1
        except Exception as e:
            print(f"Error processing file {file}: {e}")

    print(f"Processed {processed_files}/{total_files} files.")

    with open('Training.json', 'w', encoding="utf-8") as json_file:  
        json.dump(json_data, json_file, ensure_ascii=False)

AnnotationMaker("RESUMEvalidation//")


 50%|██████████████████████████████████████████                                          | 1/2 [00:08<00:08,  8.13s/it]

(0, 22) MOHAMED AMINE BOUJRADA
(23, 65) Hay Omar Ben khattab Rue 5 N°4, Casablanca
(66, 96) Date de naissance : 25-04-1987
(97, 121) Mobile : 212 658 550 654
(122, 154) E-Mail : amineboujrada@gmail.com
(245, 271)  Mars 2013 - aujourd’hui :
(272, 319) Analyste Programmeur, Softsevenart , Casablanca
(322, 391) Étude et réalisation des spécifications fonctionnelles et techniques.
(394, 462) Développement de nouvelles sites Web dynamiques et applications web.
(465, 525) Création et développement des composants PHP5 orienté objet.
(528, 568) Maintenance des applications existantes.
(634, 685) Migration vers Zend Framework 1.12 depuis mars 2014
(688, 717) Juillet 2011 - Octobre 2012 :
(718, 758) Analyste Programmeur, Siway , Casablanca
(761, 800) Développement des sites Web dynamiques.
(465, 525) Création et développement des composants PHP5 orienté objet.
(528, 568) Maintenance des applications existantes.
(909, 937) septembre 2010 – Juin 2011 :
(938, 980) Analyste Programmeur, Merkatis, Ca

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:25<00:00, 12.82s/it]

(0, 189) Niveaux d’intervention Définition des besoins. Rédaction des spécificationstechniques. Développement. Réalisation et tests. Recette et suivi des mises à jour. Reporting. Reprise des données
(190, 201) Competences
(202, 940) Environnement de développement :Oracle retail (RMS),Oracle Developer Suite ,oracle reports,Crystal Report / MS Visual Studio (.NET 1.1/2.0/3.5), Eclipse, Langages: Pl/SQL ,C/C++,pro*C, Visual Basic, Java, ASP, PHP , DotNET (VB.Net, C#, ASP.Net), T-SQL Serveur web: Apache 1.3 / 2.0, IIS, Tomcat 6 Langages scripts: HTML, ASP, PHP, XML, Javascript Méthode d’analyse: MERISE, UML SGBD: Access, SQL Server, MySql, GEMBASE, Oracle 9i/10g Open source pour le monitoring: Grafana/Telegraf/Influxdb. Réseaux & Telecom: ODBC | Ethernet | SMTP | TCP/IP Gestion de projet: Ms project, OpenWorkbench, Gantt Autres: Ms Office, PhotoShop, Power AMC, Dreamweaver, Windows Server 2003, UNIX , Intelligence Artificielle, notions de base dans la monétique
(941, 1101) Langues Arabe :p

String data saved to Data.json


In [36]:
import json
import spacy
from spacy.tokens import DocBin

nlp = spacy.blank("fr")
db = DocBin()


with open("Training.json", "r", encoding="utf-8") as f:
    dataV = json.load(f)


for text, annot in dataV:
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        print(start,end,label)
        print(span)
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents
    db.add(doc)
db.to_disk("./Training.spacy")



0 22 NAME
MOHAMED AMINE BOUJRADA
23 65 LOCATION
Hay Omar Ben khattab Rue 5 N°4, Casablanca
66 96 DATES
Date de naissance : 25-04-1987
97 121 DATES
Mobile : 212 658 550 654
122 154 DATES
E-Mail : amineboujrada@gmail.com
245 271 DATES
Mars 2013 - aujourd’hui :
272 319 EXPERIENCE
Analyste Programmeur, Softsevenart , Casablanca
322 391 EXPERIENCE
Étude et réalisation des spécifications fonctionnelles et techniques.
394 462 EXPERIENCE
Développement de nouvelles sites Web dynamiques et applications web.
465 525 EXPERIENCE
Création et développement des composants PHP5 orienté objet.
528 568 EXPERIENCE
Maintenance des applications existantes.
634 685 EXPERIENCE
Migration vers Zend Framework 1.12 depuis mars 2014
688 717 DATES
Juillet 2011 - Octobre 2012 :
718 758 EXPERIENCE
Analyste Programmeur, Siway , Casablanca
761 800 EXPERIENCE
Développement des sites Web dynamiques.
909 937 DATES
septembre 2010 – Juin 2011 :
938 980 EXPERIENCE
Analyste Programmeur, Merkatis, Casablanca
394 472 EXPERIENCE

ValueError: [E1010] Unable to set entity information for token 95 which is included in more than one span in entities, blocked, missing or outside.